In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
trainx = pd.read_csv('train.csv')
testx = pd.read_csv('test.csv')
trainy = trainx['Survived']
testy1 = pd.read_csv('gender_submission.csv')
trainx.drop(['Name', 'Ticket', 'Cabin'], axis = 1, inplace=True)
testy = testy1['Survived']
testx = testx.fillna(testx.Fare.mean())

In [3]:
# Really useful//
trainx.Sex = trainx['Sex'].astype("category").cat.codes
testx.Sex = testx['Sex'].astype("category").cat.codes
trainx.Embarked = trainx['Embarked'].astype("category").cat.codes
# now we will find the coorrelation
train= trainx.fillna({'Age': 12, 'Embarked': '1'})
train[train.columns[1:]].corr()['Survived'][:]

Survived    1.000000
Pclass     -0.338481
Sex        -0.543351
Age        -0.017315
SibSp      -0.035322
Parch       0.081629
Fare        0.257307
Embarked   -0.176509
Name: Survived, dtype: float64

In [4]:
trainx[trainx.columns[1:]].corr()['Survived'][:]

Survived    1.000000
Pclass     -0.338481
Sex        -0.543351
Age        -0.077221
SibSp      -0.035322
Parch       0.081629
Fare        0.257307
Embarked   -0.176509
Name: Survived, dtype: float64

In [5]:
trainx = trainx.drop(['Survived', 'Age', 'Embarked'], axis=1)
testx = testx.drop(['Name', 'Ticket', 'Cabin', 'Age', 'Embarked' ],axis=1)

In [6]:
trainx = np.asarray(trainx)
trainy = np.asarray(trainy)
testx = np.asarray(testx)
testy = np.asarray(testy)

In [7]:
# we normalize the dataset:
from sklearn import preprocessing
trainx = preprocessing.StandardScaler().fit(trainx).transform(trainx)
testx = preprocessing.StandardScaler().fit(testx).transform(testx)

In [8]:
print(testx.shape)
print(testy.shape)

(418, 6)
(418,)


 This function implements logistic regression and can use different numerical optimizers to find parameters, including ‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’ solvers. You can find extensive information about the pros and cons of these optimizers if you search it in internet.

The version of Logistic Regression in Scikit-learn, support regularization. Regularization is a technique used to solve the overfitting problem in machine learning models. C parameter indicates inverse of regularization strength which must be a positive float. Smaller values specify stronger regularization. Now lets fit our model with train set:

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(trainx,trainy)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
pred = LR.predict(testx)
pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

predict_proba returns estimates for all classes, ordered by the label of classes. So, the first column is the probability of class 1, P(Y=1|X), and second column is probability of class 0, P(Y=0|X):

In [11]:
yhat_prob = LR.predict_proba(testx)
yhat_prob

array([[0.77962854, 0.22037146],
       [0.46447538, 0.53552462],
       [0.69317934, 0.30682066],
       [0.77915323, 0.22084677],
       [0.45945337, 0.54054663],
       [0.77883201, 0.22116799],
       [0.43161755, 0.56838245],
       [0.70672884, 0.29327116],
       [0.43193525, 0.56806475],
       [0.81327488, 0.18672512],
       [0.77957496, 0.22042504],
       [0.57908477, 0.42091523],
       [0.21851912, 0.78148088],
       [0.70933551, 0.29066449],
       [0.23057377, 0.76942623],
       [0.34234332, 0.65765668],
       [0.69128887, 0.30871113],
       [0.77994206, 0.22005794],
       [0.46368054, 0.53631946],
       [0.4319135 , 0.5680865 ],
       [0.58420442, 0.41579558],
       [0.78173243, 0.21826757],
       [0.22459154, 0.77540846],
       [0.54981231, 0.45018769],
       [0.13304312, 0.86695688],
       [0.79773019, 0.20226981],
       [0.20727264, 0.79272736],
       [0.77992607, 0.22007393],
       [0.57544005, 0.42455995],
       [0.81447574, 0.18552426],
       [0.

In [16]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(testy, pred)

0.9736842105263158

In [19]:
new = pd.Series(pred)
sub = pd.DataFrame({'PassengerId': testy1['PassengerId'], 'Survived': new})
sub.to_csv(r'C:\Users\Sana Shah\Desktop\Practice\Logistic Regression\Submission.csv', index=False)